In [1]:
import re
import json
import os
from file_separator import split_up_file

In [2]:
# for model in os.listdir('views'):
#     view = 'sf__leads.view.lkml'
#     view_folder = view.split('.')[0]
#     split_up_model(f'views/{view}', f"{view_folder}", file_type='view')
    

In [15]:
view_list = []
for view in os.listdir('../views/sf__accounts/'):
    with open('../views/sf__accounts/'+view, 'r') as f:
        view_list.append(json.load(f))
        


In [1]:
import os

In [23]:
os.walk('.')

<generator object walk at 0x1036faf50>

In [30]:
set(os.listdir('../views')) - set(next(os.walk('../views'))[1])

{'sf__accounts.view.lkml'}

# view type 1: extends, but with explicit `sql_table_name`

In [14]:
def sourcing_table(view_path):
    
    with open(view_path, 'r') as f:
        view = json.load(f)
        
    view_source = {'view_name': view['view_name']}
    line_dic = {}
    for line in view['view']:
        line_list = list(filter(None, line.split(' ')))
        if len(line_list) > 0 and (bool(re.search('extends', line_list[0])) or bool(re.search('sql_table_name', line_list[0]))):
            line_dic[line_list[0]] = line_list[1]
    
    override_key_list = [bool(re.search('sql_table_name', x)) for x in line_dic.keys()]
    
    if (len(override_key_list) > 1 and bool(override_key_list)) or (len(override_key_list) == 1 and override_key_list[0] == True):
        view_source['source_table'] = line_dic['sql_table_name:']
    else:
        view_source['source_table'] = line_dic['extends:']

    return view_source
        
        

In [12]:
with open('../views/sf__accounts/snowflake_sf__accounts.json', 'r') as f:
    view_1 = json.load(f)
view_1['view']

['view: snowflake_sf__accounts {',
 '  extends: [sf__accounts]',
 '  sql_table_name: SEGMENT.SALESFORCE.ACCOUNTS ;;',
 '}',
 '']

In [15]:
sourcing_table(view_path='../views/sf__accounts/snowflake_sf__accounts.json')

{'view_name': 'snowflake_sf__accounts',
 'source_table': 'SEGMENT.SALESFORCE.ACCOUNTS'}

# view type 2: extends from another view, without explicit table

In [2]:
with open('../views/sf__accounts/sf__accounts.json', 'r') as f:
    view_2 = json.load(f)
view_2['view']

['view: sf__accounts {',
 '  extends: [sfbase__accounts]',
 '',
 '  filter: created_date_condition {',
 '    type: date',
 '  }',
 '',
 '  dimension: id {',
 '    #X# extends:"account_id_c"',
 '  }',
 '',
 '  dimension: is_in_created_date_filter {',
 '    type: yesno',
 '    sql: {% condition created_date_condition %} ${created_date} {% endcondition %}',
 '      ;;',
 '  }',
 '',
 '  measure: accounts_in_created_date_filter {',
 '    type: count',
 '    filters: {',
 '      field: is_in_created_date_filter',
 '      value: "yes"',
 '    }',
 '  }',
 '',
 '  measure: percent_of_accounts {',
 '    type: percent_of_total',
 '    sql: ${count} ;;',
 '  }',
 '',
 '  measure: count_customers {',
 '    type: count',
 '    filters: {',
 '      field: type',
 '      value: "\\"Customer\\""',
 '    }',
 '    drill_fields: [detail*]',
 '  }',
 '}',
 '']

In [32]:
view_source = {'view_name': view_2['view_name']}
line_dic = {}
for line in view_2['view']:
    line_list = list(filter(None, line.split(' ')))
    if len(line_list) > 0 and (bool(re.search('extends', line_list[0])) or bool(re.search('sql_table_name', line_list[0]))):
        line_dic[line_list[0]] = line_list[1]

override_key_list = [bool(re.search('sql_table_name', x)) for x in line_dic.keys()]


In [35]:
line_list


[]

In [9]:
sourcing_table('../views/sf__accounts/sf__accounts.json')

{'view_name': 'sf__accounts', 'source_table': '[sfbase__accounts]'}

# view type 3: directly from `sql_table_name`

In [9]:
with open('../views/sf__accounts/sfbase__accounts.json', 'r') as f:
    view_3 = json.load(f)
view_3['view']

['view: sfbase__accounts {',
 '  sql_table_name: salesforce.accounts ;;',
 '',
 '  dimension: account_band {',
 '    type: string',
 '    description: "Standardized revenue band for this account"',
 '    sql: ${TABLE}.account_band_c ;;',
 '  }',
 '',
 '  dimension: account_id {',
 '    description: "Case-sensitive old-style SFDC account id.  Recommend using ID instead"',
 '    hidden: no',
 '    type: string',
 '    sql: ${TABLE}.account_id_c ;;',
 '  }',
 '',
 '  dimension: account_next_step {',
 '    type: string',
 '    sql: ${TABLE}.account_next_step_c ;;',
 '  }',
 '',
 '  dimension: account_notes {',
 '    type: string',
 '    sql: ${TABLE}.account_notes_c ;;',
 '  }',
 '',
 '  dimension: account_record {',
 '    type: string',
 '    sql: ${TABLE}.account_record_c ;;',
 '  }',
 '',
 '  dimension: account_time_zone {',
 '    type: string',
 '    sql: ${TABLE}.account_time_zone_c ;;',
 '  }',
 '',
 '  dimension: acct_no {',
 '    type: number',
 '    sql: ${TABLE}.acct_no_c ;;',
 '

In [10]:
sourcing_table('../views/sf__accounts/sfbase__accounts.json')

{'view_name': 'sfbase__accounts', 'source_table': 'salesforce.accounts'}

# view type 4: NDT from an explore

In [3]:
with open('../views/sf__accounts/total_active_node_count.json', 'r') as f:
    view_4 = json.load(f)
view_4['view']

['view: total_active_node_count {',
 '  derived_table: {',
 '    explore_source: sf__opportunities {',
 '      column: id { field: sf__accounts.id }',
 '      column: name { field: sf__accounts.name }',
 '      column: total_quantity { field: opportunity_products.total_quantity }',
 '      column: node_type { field: opportunity_products.node_type }',
 '      filters: {',
 '        field: opportunity_products.end_date',
 '        value: "after 1 days from now"',
 '      }',
 '      filters: {',
 '        field: opportunity_products.node_type',
 '        value: "-None"',
 '      }',
 '      filters: {',
 '        field: opportunity_products.product_family',
 '        value: "Subscription"',
 '      }',
 '      filters: {',
 '        field: sf__opportunities.is_won',
 '        value: "Yes"',
 '      }',
 '    }',
 '  }',
 '  parameter: bucket_size {',
 '    type: number',
 '    default_value: "5"',
 '  }',
 '',
 '  dimension: id {',
 '    label: "Accounts ID"',
 '    primary_key: yes',
 '

In [86]:
def get_derived_table(view_list):
    
    derived_list = []
    for num, line in enumerate(view_list):

        line_list = list(filter(None, line.split(' ')))
        derived_list.append(line_list)
        end_of_derived_table = sum([bool(re.search('dimension', s) or re.search('parameter', s) or re.search('measure', s)) for s in line_list])

        if end_of_derived_table > 0:
            break
            
    return derived_list[1:-1]
        

In [87]:
derived_list = get_derived_table(view_4['view'])

In [89]:
view_source = {'view_name': view_4['view_name']}
view_source['derived_table'] = derived_list

In [90]:
view_source

{'view_name': 'total_active_node_count',
 'derived_table': [['derived_table:', '{'],
  ['explore_source:', 'sf__opportunities', '{'],
  ['column:', 'id', '{', 'field:', 'sf__accounts.id', '}'],
  ['column:', 'name', '{', 'field:', 'sf__accounts.name', '}'],
  ['column:',
   'total_quantity',
   '{',
   'field:',
   'opportunity_products.total_quantity',
   '}'],
  ['column:',
   'node_type',
   '{',
   'field:',
   'opportunity_products.node_type',
   '}'],
  ['filters:', '{'],
  ['field:', 'opportunity_products.end_date'],
  ['value:', '"after', '1', 'days', 'from', 'now"'],
  ['}'],
  ['filters:', '{'],
  ['field:', 'opportunity_products.node_type'],
  ['value:', '"-None"'],
  ['}'],
  ['filters:', '{'],
  ['field:', 'opportunity_products.product_family'],
  ['value:', '"Subscription"'],
  ['}'],
  ['filters:', '{'],
  ['field:', 'sf__opportunities.is_won'],
  ['value:', '"Yes"'],
  ['}'],
  ['}'],
  ['}']]}

# test script

In [44]:
def sourcing_table(view_path):

    with open(view_path, 'r') as f:    
        view = json.load(f)

    view_source = {'view_name': view['view_name']}
    line_dict = {}
    
    for line in view['view']:
        line_list = list(filter(None, line.split(' ')))
        if len(line_list) > 0 and (bool(re.search('extends', line_list[0])) \
                                   or bool(re.search('sql_table_name', line_list[0]))):
            line_dict[line_list[0]] = line_list[1]
        elif len(line_list) > 0 and bool(re.search('derived_table', line_list[0])):
            view_source['view_type'] = 'derived_table'

    override_key_list = [bool(re.search('sql_table_name', x)) for x in line_dic.keys()]

    if sum(override_key_list) >= 1:
        view_source['source_table'] = line_dic['sql_table_name:']
        view_source['view_type'] = 'sql_table_name'
    elif sum(override_key_list) == 0:
        view_source['source_table'] = line_dic['extends:']
        view_source['view_type'] = 'extension'

    if view_source['view_type'] == 'derived_table':
        derived_list = get_derived_table(view['view'])
        view_source['derived_table'] = derived_list
    
    return view_source
        

In [73]:
import re
import json
import os


def get_derived_table(view_list):
    
    derived_list = []
    for num, line in enumerate(view_list):

        line_list = list(filter(None, line.split(' ')))
        derived_list.append(line_list)
        end_of_derived_table = sum([bool(re.search('dimension', s) or re.search('parameter', s) or re.search('measure', s)) for s in line_list])

        if end_of_derived_table > 0:
            break
            
    return derived_list[1:-1]
        

def sourcing_table(view_path):
    """
    This function extracts the source view/table from the `sql_table_name` parameter, or from the `extends` parameter.

    :param view_path: the path of the view file

    :type view_path: string
    
    :return: json payload of view name and the source table.
    """

    with open(view_path, 'r') as f:    
        view = json.load(f)

    view_source = {'view_name': view['view_name']}
    line_dict = {}
    
    for line in view['view']:
        line_list = list(filter(None, line.split(' ')))
        if len(line_list) > 0 and (bool(re.search('extends', line_list[0])) \
                                   or bool(re.search('sql_table_name', line_list[0]))):
            line_dict[line_list[0]] = line_list[1]
        elif len(line_list) > 0 and bool(re.search('derived_table', line_list[0])):
            view_source['view_type'] = 'derived_table'

    override_key_list = [bool(re.search('sql_table_name', x)) for x in line_dict.keys()]

    if len(override_key_list) >= 1 and sum(override_key_list) >= 1:
        view_source['source_table'] = line_dict['sql_table_name:']
        view_source['view_type'] = 'sql_table_name'
    elif len(override_key_list) == 1 and sum(override_key_list) == 0:
        view_source['source_table'] = line_dict['extends:']
        view_source['view_type'] = 'extension'

    if view_source['view_type'] == 'derived_table':
        derived_list = get_derived_table(view['view'])
        view_source['derived_table'] = derived_list
    
    return view_source
        

In [74]:
dir_path = '../views/sf__accounts/'
view_path = [dir_path+'sf__accounts.json', dir_path+'snowflake_sf__accounts.json', dir_path+'sfbase__accounts.json',\
            dir_path+'total_active_node_count.json']

In [75]:
for view in view_path:
    print(view, '\n', sourcing_table(view))

../views/sf__accounts/sf__accounts.json 
 {'view_name': 'sf__accounts', 'source_table': '[sfbase__accounts]', 'view_type': 'extension'}
../views/sf__accounts/snowflake_sf__accounts.json 
 {'view_name': 'snowflake_sf__accounts', 'source_table': 'SEGMENT.SALESFORCE.ACCOUNTS', 'view_type': 'sql_table_name'}
../views/sf__accounts/sfbase__accounts.json 
 {'view_name': 'sfbase__accounts', 'source_table': 'salesforce.accounts', 'view_type': 'sql_table_name'}
../views/sf__accounts/total_active_node_count.json 
 {'view_name': 'total_active_node_count', 'view_type': 'derived_table', 'derived_table': [['derived_table:', '{'], ['explore_source:', 'sf__opportunities', '{'], ['column:', 'id', '{', 'field:', 'sf__accounts.id', '}'], ['column:', 'name', '{', 'field:', 'sf__accounts.name', '}'], ['column:', 'total_quantity', '{', 'field:', 'opportunity_products.total_quantity', '}'], ['column:', 'node_type', '{', 'field:', 'opportunity_products.node_type', '}'], ['filters:', '{'], ['field:', 'opportuni